# RecSys Model Ensemble

##### Using voting 

In [1]:
import os
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, StandardScaler

def ensemble_recommendations(folder_path, final_K, scaling_method='standard', weights=None): # weights : 가중치를 리스트 형태로
    # 주어진 폴더에서 .csv 화일 목록 가져오기
    csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
    
    # 각 화일을 데이터프레임으로 읽어오기
    dfs = [pd.read_csv(os.path.join(folder_path, csv_file)) for csv_file in csv_files]
    
    # 가중치가 주어지지 않은 경우, 모든 알고리즘에 동일한 가중치 (1) 부여
    if weights is None:
        weights = [1] * len(dfs)
    
    # 스케일러 초기화
    if scaling_method == 'minmax':
        scaler = MinMaxScaler()
    elif scaling_method == 'standard':
        scaler = StandardScaler()
    
    # 모든 데이터프레임들을 하나로 합치기 전에 각 데이터프레임마다 prediction 값을 정규화
    normalized_dfs = []
    for i, df in enumerate(dfs):
        df['prediction'] = scaler.fit_transform(df['prediction'].values.reshape(-1, 1)).flatten()
        # 가중치 적용
        df['prediction'] *= weights[i]
        normalized_dfs.append(df)

    # 정규화 및 가중치 적용된 데이터프레임들을 하나로 합치기
    combined_df = pd.concat(normalized_dfs, ignore_index=True)
    
    # prediction 값을 추후 count해줄 컬럼 추가
    combined_df['prediction_cnt'] =  combined_df['prediction']
    
    #resume_seq와 recruitment_seq를 기준으로 prediction 값을 그룹화하고 평균 계산
    counted_predictions = combined_df.groupby(['resume_seq', 'recruitment_seq']).aggregate({'prediction':'mean','prediction_cnt':'count'}).reset_index()
    
    # 자주 등장한 값 내림차순 정렬 후, 같은 값에 대해서 prediction평균 내림차순 순
    top_k = (
        counted_predictions
        .groupby('resume_seq')
        .apply(lambda x: x.sort_values(by=['prediction_cnt','prediction'],ascending=[False,False]).head(5))
        .reset_index(drop=True)
    )

    print(top_k)
    
    return top_k

In [2]:
# 함수 호출
recommendations = ensemble_recommendations('./preds', final_K=5)

# 결과 저장
t = pd.Timestamp.now()
fname = f"submit_ensemble_{t.month:02}{t.day:02}{t.hour:02}{t.minute:02}.csv"
recommendations[['resume_seq', 'recruitment_seq']].to_csv(fname, index=False)

      resume_seq recruitment_seq  prediction  prediction_cnt
0         U00001          R01528    0.846749               6
1         U00001          R00165    0.820276               6
2         U00001          R06276    0.303367               6
3         U00001          R03811    0.293427               6
4         U00001          R02888    0.232514               6
...          ...             ...         ...             ...
42405     U08482          R04602    1.673135               6
42406     U08482          R00712    0.370244               6
42407     U08482          R00473    0.272431               6
42408     U08482          R05461   -0.016783               6
42409     U08482          R02524   -0.114000               6

[42410 rows x 4 columns]
